In [3]:
# from podcast_ad_skipper.Spectrogram_Conversion import create_spectrogram
import pandas as pd
from podcast_ad_skipper.google_cloud import auth_gc, call_bq_client, append_arrays_to_bq
import librosa
from podcast_ad_skipper.params import *

from google.auth.exceptions import GoogleAuthError
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery


In [4]:
storage_client = auth_gc()

Authenticated successfully! ✅


In [5]:
prefixes =GCP_PREFIXES[0]
bucket_name = BUCKET_NAME

In [6]:
def retrieve_files_in_folder(bucket_name, prefixes):
    file_list = []
    bucket = storage_client.bucket(bucket_name)
    file_list_google_object = bucket.list_blobs(prefix=prefixes)
    for file in file_list_google_object:
        file_list.append(file)
    return file_list


In [7]:
file_list = retrieve_files_in_folder(bucket_name, prefixes)


In [8]:
type(file_list)

list

In [9]:
def open_gcs_file(file):
    audio_file = file.open('rb')
    return audio_file

In [10]:
open_file = open_gcs_file(file_list[0])

In [11]:
def create_spectrogram(open_file):
    #y is an array representing the frequency of the audio signal at each sample.
    #sr is the sampling rate (samples per second),
    y, sr = librosa.load(open_file)
    spectrogram = librosa.stft(y)

    #Transforms to decibel scale (logarithmic), which gives more emphasis to amplitude changes in high volumes
    spectrogram_db = librosa.amplitude_to_db(abs(spectrogram))
    return spectrogram_db

In [12]:
create_spectrogram(open_file)

array([[-10.198136, -15.113926, -29.613918, ..., -29.926647, -24.283981,
        -18.033857],
       [-10.190883, -19.369791, -25.763617, ..., -27.13015 , -23.894823,
        -19.20064 ],
       [ -9.43372 , -15.497618, -26.510141, ..., -30.378845, -29.13692 ,
        -23.978172],
       ...,
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ]], dtype=float32)

In [13]:
spectrogram_list=[]

In [14]:
#main.py


def we_have_to_Change_the_name(bucket_name, prefixes):
    file_list = retrieve_files_in_folder(bucket_name, prefixes)
    for file in file_list[:6]:
        open_file = open_gcs_file(file)
        spectrogram_db = create_spectrogram(open_file)
        spectrogram_list.append(spectrogram_db)


In [15]:
we_have_to_Change_the_name(bucket_name, prefixes)

In [16]:
len(spectrogram_list)

6

In [17]:
spectrogram_list[0]

array([[-10.198136, -15.113926, -29.613918, ..., -29.926647, -24.283981,
        -18.033857],
       [-10.190883, -19.369791, -25.763617, ..., -27.13015 , -23.894823,
        -19.20064 ],
       [ -9.43372 , -15.497618, -26.510141, ..., -30.378845, -29.13692 ,
        -23.978172],
       ...,
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ],
       [-38.53384 , -38.53384 , -38.53384 , ..., -38.53384 , -38.53384 ,
        -38.53384 ]], dtype=float32)

In [18]:
bq_client = call_bq_client()


Authenticated successfully with BigQuery! ✅


In [19]:
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}"

In [20]:
np_array = spectrogram_list[0]

In [21]:
def append_arrays_to_bq(bq_client, np_array, table_id):
    # Create df out of the np arrays, to upload to bq
    columns = [f'col_{i}' for i in range(np_array.shape[1])]
    df = pd.DataFrame(np_array, columns=columns)

    # Use WRITE_APPEND to add to the existing table
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")

    job = bq_client.load_table_from_dataframe(df, table_id, job_config=job_config)

    print(job.result())

    print(f"Appended rows to {table_id}")


In [22]:
append_arrays_to_bq(bq_client, np_array, table_id)

LoadJob<project=podcast-ad-skipper, location=EU, id=b87efc79-fe7d-439e-8ce3-4274893a952b>
Appended rows to podcast-ad-skipper.Numpy_Arrays_Dataset.Numpy_Arrays_Table
